# DGD+TM算法+极小值攻击

In [12]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [13]:
#定义变量 
agent_resource_quantity_robust1 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust1 = 0.6
dual_variable_robust1 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust1= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust1=[]#绘图横坐标
A_robust1 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")
neighbor_robust1_help=[]
a_robust1=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust1=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
q_robust1=[] #TM鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量

In [14]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust1*100)):
    dual_variable_robust1[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust1*100)),100):
    dual_variable_robust1[i][0]=-600
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust1*100)):
    agent_resource_quantity_robust1[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust1=0
for i in range (0,100-int(proportion_robust1*100)):
    sum_robust1 = sum_robust1 + agent_resource_quantity_robust1[i][0]
average_resource_honest_robust1[0][0]=sum_robust1/(100-int(proportion_robust1*100))

#诚实用户平均成本初始化
sum_robust1=0
for i in range(0,100-int(proportion_robust1*100)):
    sum_robust1=sum_robust1+a_robust1[i]*(agent_resource_quantity_robust1[i][0]-b_robust1[i])*(agent_resource_quantity_robust1[i][0]-b_robust1[i])
average_cost_honest_robust1[0][0]=sum_robust1/(100-int(proportion_robust1*100))

iteration_robust1.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust1[i][j]!=0:
            neighbor_robust1_help.append(j)
neighbor_robust1=np.array(neighbor_robust1_help).reshape(100,15)

#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust1_help=0
    for j in range(0,15):
        if neighbor_robust1[i][j]==94 or neighbor_robust1[i][j]==95 or neighbor_robust1[i][j]==96 or neighbor_robust1[i][j]==97 or neighbor_robust1[i][j]==98 or neighbor_robust1[i][j]==99:
            q_robust1_help=q_robust1_help+1
    q_robust1.append(q_robust1_help) 

In [4]:
#DGD算法 
def Robust_DGD_TM_robust1(a_robust1,b_robust1,agent_resource_quantity_robust1,average_resource_honest_robust1,average_cost_honest_robust1,iteration_update_robust1,dual_variable_robust1,A_robust1,iteration_robust1):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+TM+小值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust1[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust1[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust1[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust1[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust1)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust1.append(k)
                
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust1=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust1*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust1.append(dual_variable_robust1[i][k-1]-iteration_update_robust1*((50/100)-(1/100)*agent_resource_quantity_robust1[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust1*100)),100):
            dual_variable_intermediate_robust1.append(dual_variable_robust1[i][k-1])
        #阶段二：TM鲁棒聚合 每个用户聚合自身以及邻居的对偶值    
        for i in range (0,100-int(proportion_robust1*100)):
            mix_sum_dual_robust1=0
            sort_list_robust1=[]
            sum_robust1=0
            for j in range (0,15):
                sort_list_robust1.append(dual_variable_intermediate_robust1[neighbor_robust1[i,j]])
            sort_list_robust1.sort()
            #剔除最大的q个数据和最小的q个数据
            for j in range(0,int(q_robust1[i])):
                del sort_list_robust1[14-j]
            for j in range(0,int(q_robust1[i])):
                del sort_list_robust1[0]     
            for j in range(0,len(sort_list_robust1)):
                mix_sum_dual_robust1=mix_sum_dual_robust1+sort_list_robust1[j]
            #诚实用户将剩下的数据和自身的数据做平均操作
            dual_variable_robust1[i][k]=(1/(16-2*q_robust1[i]))*(dual_variable_robust1[i][k-1]+mix_sum_dual_robust1)
        
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust1*100),100):
            dual_variable_robust1[i][k]=dual_variable_robust1[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust1*100)):
            agent_resource_quantity_robust1[i][k]=(dual_variable_robust1[i][k]-2*a_robust1[i]*b_robust1[i])/(-2*a_robust1[i])
            #原变量的投影操作
            if agent_resource_quantity_robust1[i][k] < 0:
                agent_resource_quantity_robust1[i][k]=0
            elif agent_resource_quantity_robust1[i][k] > 100: 
                agent_resource_quantity_robust1[i][k]=100   
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust1*100),100):
            agent_resource_quantity_robust1[i][k]=agent_resource_quantity_robust1[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust1=0
        for i in range (0,100-int(proportion_robust1*100)):
            sum1_robust1 = sum1_robust1 + agent_resource_quantity_robust1[i][k]
        average_resource_honest_robust1[0][k]=sum1_robust1/(100-int(proportion_robust1*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust1=0
        for i in range(0,100-int(proportion_robust1*100)):
            sum2_robust1=sum2_robust1+a_robust1[i]*(agent_resource_quantity_robust1[i][k]-b_robust1[i])*(agent_resource_quantity_robust1[i][k]-b_robust1[i])
        average_cost_honest_robust1[0][k]=sum2_robust1/(100-int(proportion_robust1*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust1[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust1[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust1[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust1[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust1)
            print("****************************************************************************")
        
        iteration_update_robust1=1/(pow(k+1,0.1)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [5]:
Robust_DGD_TM_robust1(a_robust1,b_robust1,agent_resource_quantity_robust1,average_resource_honest_robust1,average_cost_honest_robust1,iteration_update_robust1,dual_variable_robust1,A_robust1,iteration_robust1)

****************************DGD算法+TM+小值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 57.319758690223814
agent 2 在 3999 次迭代中的资源申请量为: 57.13810584796238
agent 3 在 3999 次迭代中的资源申请量为: 61.869659901186914
agent 4 在 3999 次迭代中的资源申请量为: 38.68793832297003
agent 5 在 3999 次迭代中的资源申请量为: 58.70897374974778
agent 6 在 3999 次迭代中的资源申请量为: 56.436441465040375
agent 7 在 3999 次迭代中的资源申请量为: 66.96144369059553
agent 8 在 3999 次迭代中的资源申请量为: 47.25075427397654
agent 9 在 3999 次迭代中的资源申请量为: 66.96064081498959
agent 10 在 3999 次迭代中的资源申请量为: 55.309428426280085
agent 11 在 3999 次迭代中的资源申请量为: 66.80267697852392
agent 12 在 3999 次迭代中的资源申请量为: 39.17322805520621
agent 13 在 3999 次迭代中的资源申请量为: 50.71326772702402
agent 14 在 3999 次迭代中的资源申请量为: 42.17447222290589
agent 15 在 3999 次迭代中的资源申请量为: 43.32933506064631
agent 16 在 3999 次迭代中的资源申请量为: 50.2294145382329
agent 17 在 3999 次迭代中的资源申请量为: 51.10525046068
agent 18 在 3999 次迭代中的资源申请量为: 47.4643303264253
agent 19 在 3999 次迭代中的资源申请量为: 45.683607632827126
agent 20 在 3999 次迭代中的资源申请量为: 51.798232588813
agent 21 在 3999 次迭代中的资源申请量为: 71.33371280161958
agent 22 在

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 57.41873726943906
agent 2 在 5999 次迭代中的资源申请量为: 57.23685679758693
agent 3 在 5999 次迭代中的资源申请量为: 61.97836528883028
agent 4 在 5999 次迭代中的资源申请量为: 38.75512673210574
agent 5 在 5999 次迭代中的资源申请量为: 58.81030105261132
agent 6 在 5999 次迭代中的资源申请量为: 56.53366319833356
agent 7 在 5999 次迭代中的资源申请量为: 67.07972585975274
agent 8 在 5999 次迭代中的资源申请量为: 47.332134471783135
agent 9 在 5999 次迭代中的资源申请量为: 67.07764439718932
agent 10 在 5999 次迭代中的资源申请量为: 55.40582987694274
agent 11 在 5999 次迭代中的资源申请量为: 66.92166253667116
agent 12 在 5999 次迭代中的资源申请量为: 39.240506843844294
agent 13 在 5999 次迭代中的资源申请量为: 50.8028628049651
agent 14 在 5999 次迭代中的资源申请量为: 42.24665851014231
agent 15 在 5999 次迭代中的资源申请量为: 43.403518095356375
agent 16 在 5999 次迭代中的资源申请量为: 50.31573896992776
agent 17 在 5999 次迭代中的资源申请量为: 51.192628520244135
agent 18 在 5999 次迭代中的资源申请量为: 47.54625812777924
agent 19 在 5999 次迭代中的资源申请量为: 45.76357367523892
agent 20 在 5999 次迭代中的资源申请量为: 51.8869328329492
agent 21 在 5999 次迭代中的资源申请量为: 71.45807481367703
agent 

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 57.42492309303879
agent 2 在 7999 次迭代中的资源申请量为: 57.24335804327243
agent 3 在 7999 次迭代中的资源申请量为: 61.985322841536764
agent 4 在 7999 次迭代中的资源申请量为: 38.75943559622975
agent 5 在 7999 次迭代中的资源申请量为: 58.816729736589956
agent 6 在 7999 次迭代中的资源申请量为: 56.53985661075785
agent 7 在 7999 次迭代中的资源申请量为: 67.08714940363988
agent 8 在 7999 次迭代中的资源申请量为: 47.337253120469306
agent 9 在 7999 次迭代中的资源申请量为: 67.08508137597654
agent 10 在 7999 次迭代中的资源申请量为: 55.412227117356416
agent 11 在 7999 次迭代中的资源申请量为: 66.929044839552
agent 12 在 7999 次迭代中的资源申请量为: 39.24462402368506
agent 13 在 7999 次迭代中的资源申请量为: 50.80849757206406
agent 14 在 7999 次迭代中的资源申请量为: 42.25132737444938
agent 15 在 7999 次迭代中的资源申请量为: 43.40814334594171
agent 16 在 7999 次迭代中的资源申请量为: 50.32102173429618
agent 17 在 7999 次迭代中的资源申请量为: 51.19813906694763
agent 18 在 7999 次迭代中的资源申请量为: 47.551393165394025
agent 19 在 7999 次迭代中的资源申请量为: 45.768501700737055
agent 20 在 7999 次迭代中的资源申请量为: 51.89264505917495
agent 21 在 7999 次迭代中的资源申请量为: 71.46599636417068
agen

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 57.42529994873169
agent 2 在 9999 次迭代中的资源申请量为: 57.2440062164497
agent 3 在 9999 次迭代中的资源申请量为: 61.98587200740882
agent 4 在 9999 次迭代中的资源申请量为: 38.759782084967874
agent 5 在 9999 次迭代中的资源申请量为: 58.81719485825201
agent 6 在 9999 次迭代中的资源申请量为: 56.54032369637152
agent 7 在 9999 次迭代中的资源申请量为: 67.08762560678075
agent 8 在 9999 次迭代中的资源申请量为: 47.337589934548376
agent 9 在 9999 次迭代中的资源申请量为: 67.08562978043526
agent 10 在 9999 次迭代中的资源申请量为: 55.41290171321546
agent 11 在 9999 次迭代中的资源申请量为: 66.9294533783764
agent 12 在 9999 次迭代中的资源申请量为: 39.24480792142188
agent 13 在 9999 次迭代中的资源申请量为: 50.808867908114806
agent 14 在 9999 次迭代中的资源申请量为: 42.25173223322512
agent 15 在 9999 次迭代中的资源申请量为: 43.40841675990815
agent 16 在 9999 次迭代中的资源申请量为: 50.321257751442204
agent 17 在 9999 次迭代中的资源申请量为: 51.19851280162358
agent 18 在 9999 次迭代中的资源申请量为: 47.55171733998537
agent 19 在 9999 次迭代中的资源申请量为: 45.76874870969168
agent 20 在 9999 次迭代中的资源申请量为: 51.89312212427546
agent 21 在 9999 次迭代中的资源申请量为: 71.46659318140019
agent 

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 57.42494738646127
agent 2 在 51999 次迭代中的资源申请量为: 57.2455032180671
agent 3 在 51999 次迭代中的资源申请量为: 61.986403729661056
agent 4 在 51999 次迭代中的资源申请量为: 38.760158569268135
agent 5 在 51999 次迭代中的资源申请量为: 58.81737272484824
agent 6 在 51999 次迭代中的资源申请量为: 56.54063562578558
agent 7 在 51999 次迭代中的资源申请量为: 67.0873798499299
agent 8 在 51999 次迭代中的资源申请量为: 47.337483153866195
agent 9 在 51999 次迭代中的资源申请量为: 67.0859121313481
agent 10 在 51999 次迭代中的资源申请量为: 55.41464729474331
agent 11 在 51999 次迭代中的资源申请量为: 66.9287274361816
agent 12 在 51999 次迭代中的资源申请量为: 39.24407742655298
agent 13 在 51999 次迭代中的资源申请量为: 50.808747143712544
agent 14 在 51999 次迭代中的资源申请量为: 42.25235814053372
agent 15 在 51999 次迭代中的资源申请量为: 43.40809114017378
agent 16 在 51999 次迭代中的资源申请量为: 50.32032085880485
agent 17 在 51999 次迭代中的资源申请量为: 51.19848034655419
agent 18 在 51999 次迭代中的资源申请量为: 47.55150863221846
agent 19 在 51999 次迭代中的资源申请量为: 45.76807358222117
agent 20 在 51999 次迭代中的资源申请量为: 51.8937526230334
agent 21 在 51999 次迭代中的资源申请量为: 71.46

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 57.42493925497
agent 2 在 53999 次迭代中的资源申请量为: 57.24553396522082
agent 3 在 53999 次迭代中的资源申请量为: 61.98641411691627
agent 4 在 53999 次迭代中的资源申请量为: 38.760165992131334
agent 5 在 53999 次迭代中的资源申请量为: 58.81737572595762
agent 6 在 53999 次迭代中的资源申请量为: 56.54064147302932
agent 7 在 53999 次迭代中的资源申请量为: 67.08737382267134
agent 8 在 53999 次迭代中的资源申请量为: 47.33748031584677
agent 9 在 53999 次迭代中的资源申请量为: 67.08591721573882
agent 10 在 53999 次迭代中的资源申请量为: 55.414683283545315
agent 11 在 53999 次迭代中的资源申请量为: 66.92871131277732
agent 12 在 53999 次迭代中的资源申请量为: 39.24406158159847
agent 13 在 53999 次迭代中的资源申请量为: 50.80874395272559
agent 14 在 53999 次迭代中的资源申请量为: 42.25237077009297
agent 15 在 53999 次迭代中的资源申请量为: 43.40808375528322
agent 16 在 53999 次迭代中的资源申请量为: 50.32030053622049
agent 17 在 53999 次迭代中的资源申请量为: 51.19847902771492
agent 18 在 53999 次迭代中的资源申请量为: 47.55150364858248
agent 19 在 53999 次迭代中的资源申请量为: 45.76805880873303
agent 20 在 53999 次迭代中的资源申请量为: 51.89376522969504
agent 21 在 53999 次迭代中的资源申请量为: 71.46

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 57.42493144850553
agent 2 在 55999 次迭代中的资源申请量为: 57.245563484840154
agent 3 在 55999 次迭代中的资源申请量为: 61.98642408980926
agent 4 在 55999 次迭代中的资源申请量为: 38.76017311872488
agent 5 在 55999 次迭代中的资源申请量为: 58.81737860755331
agent 6 在 55999 次迭代中的资源申请量为: 56.54064708701843
agent 7 在 55999 次迭代中的资源申请量为: 67.08736803646401
agent 8 在 55999 次迭代中的资源申请量为: 47.33747759137434
agent 9 在 55999 次迭代中的资源申请量为: 67.08592209752379
agent 10 在 55999 次迭代中的资源申请量为: 55.414717835468686
agent 11 在 55999 次迭代中的资源申请量为: 66.92869583369556
agent 12 在 55999 次迭代中的资源申请量为: 39.244046369583465
agent 13 在 55999 次迭代中的资源申请量为: 50.80874088944057
agent 14 在 55999 次迭代中的资源申请量为: 42.25238289549108
agent 15 在 55999 次迭代中的资源申请量为: 43.40807666551609
agent 16 在 55999 次迭代中的资源申请量为: 50.32028102541907
agent 17 在 55999 次迭代中的资源申请量为: 51.19847776180035
agent 18 在 55999 次迭代中的资源申请量为: 47.55149886421301
agent 19 在 55999 次迭代中的资源申请量为: 45.76804462543912
agent 20 在 55999 次迭代中的资源申请量为: 51.89377733319776
agent 21 在 55999 次迭代中的资源申请量为: 7

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 57.42492394313292
agent 2 在 57999 次迭代中的资源申请量为: 57.245591867220476
agent 3 在 57999 次迭代中的资源申请量为: 61.98643367880007
agent 4 在 57999 次迭代中的资源申请量为: 38.760179970834905
agent 5 在 57999 次迭代中的资源申请量为: 58.81738137840649
agent 6 在 57999 次迭代中的资源申请量为: 56.54065248489627
agent 7 在 57999 次迭代中的资源申请量为: 67.08736247354767
agent 8 在 57999 次迭代中的资源申请量为: 47.337474972078844
agent 9 在 57999 次迭代中的资源申请量为: 67.0859267915825
agent 10 在 57999 次迭代中的资源申请量为: 55.41475105621263
agent 11 在 57999 次迭代中的资源申请量为: 66.92868095150146
agent 12 在 57999 次迭代中的资源申请量为: 39.244031743925454
agent 13 在 57999 次迭代中的资源申请量为: 50.808737944442925
agent 14 在 57999 次迭代中的资源申请量为: 42.25239455380828
agent 15 在 57999 次迭代中的资源申请量为: 43.40806984914328
agent 16 在 57999 次迭代中的资源申请量为: 50.32026226665646
agent 17 在 57999 次迭代中的资源申请量为: 51.19847654489851
agent 18 在 57999 次迭代中的资源申请量为: 47.55149426443165
agent 19 在 57999 次迭代中的资源申请量为: 45.76803098889913
agent 20 在 57999 次迭代中的资源申请量为: 51.89378897054311
agent 21 在 57999 次迭代中的资源申请量为: 

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 57.42491671743102
agent 2 在 59999 次迭代中的资源申请量为: 57.245619193183074
agent 3 在 59999 次迭代中的资源申请量为: 61.98644291115453
agent 4 在 59999 次迭代中的资源申请量为: 38.76018656796234
agent 5 在 59999 次迭代中的资源申请量为: 58.817384046369966
agent 6 在 59999 次迭代中的资源申请量为: 56.54065768200891
agent 7 在 59999 次迭代中的资源申请量为: 67.08735711802828
agent 8 在 59999 次迭代中的资源申请量为: 47.33747245047012
agent 9 在 59999 次迭代中的资源申请量为: 67.08593131123534
agent 10 在 59999 次迭代中的资源申请量为: 55.41478304038567
agent 11 在 59999 次迭代中的资源申请量为: 66.92866662374081
agent 12 在 59999 次迭代中的资源申请量为: 39.244017662931874
agent 13 在 59999 次迭代中的资源申请量为: 50.80873510930791
agent 14 在 59999 次迭代中的资源申请量为: 42.25240577823472
agent 15 在 59999 次迭代中的资源申请量为: 43.408063286717514
agent 16 在 59999 次迭代中的资源申请量为: 50.32024420646001
agent 17 在 59999 次迭代中的资源申请量为: 51.19847537350973
agent 18 在 59999 次迭代中的资源申请量为: 47.55148983610212
agent 19 在 59999 次迭代中的资源申请量为: 45.76801786023319
agent 20 在 59999 次迭代中的资源申请量为: 51.89380017485134
agent 21 在 59999 次迭代中的资源申请量为: 

In [6]:
np.savetxt('./data/02_TM_oracle_agent_resource_quantity_min_attack.txt',agent_resource_quantity_robust1,fmt='%0.8f')#用户资源
np.savetxt('./data/02_TM_oracle_average_cost_honest_min_attack.txt',average_cost_honest_robust1,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/02_TM_oracle_average_resource_honest_min_attack.txt',average_resource_honest_robust1,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/02_TM_oracle_dual_variable_min_attack.txt',dual_variable_robust1,fmt='%0.8f')#用户对偶变量

# DGD+TM算法+大值攻击

In [7]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [8]:
#定义变量 
agent_resource_quantity_robust2 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust2 = 0.6
dual_variable_robust2 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust2= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust2=[]#绘图横坐标
A_robust2 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust2_help=[]
a_robust2=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust2=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
q_robust2=[] #TM鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量

In [9]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust2*100)):
    dual_variable_robust2[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust2*100)),100):
    dual_variable_robust2[i][0]=-0.01
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust2*100)):
    agent_resource_quantity_robust2[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust2=0
for i in range (0,100-int(proportion_robust2*100)):
    sum_robust2 = sum_robust2 + agent_resource_quantity_robust2[i][0]
average_resource_honest_robust2[0][0]=sum_robust2/(100-int(proportion_robust2*100))

#诚实用户平均成本初始化
sum_robust2=0
for i in range(0,100-int(proportion_robust2*100)):
    sum_robust2=sum_robust2+a_robust2[i]*(agent_resource_quantity_robust2[i][0]-b_robust2[i])*(agent_resource_quantity_robust2[i][0]-b_robust2[i])
average_cost_honest_robust2[0][0]=sum_robust2/(100-int(proportion_robust2*100))

iteration_robust2.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust2[i][j]!=0:
            neighbor_robust2_help.append(j)
neighbor_robust2=np.array(neighbor_robust2_help).reshape(100,15)

#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust2_help=0
    for j in range(0,15):
        if neighbor_robust2[i][j]==94 or neighbor_robust2[i][j]==95 or neighbor_robust2[i][j]==96 or neighbor_robust2[i][j]==97 or neighbor_robust2[i][j]==98 or neighbor_robust2[i][j]==99:
            q_robust2_help=q_robust2_help+1
    q_robust2.append(q_robust2_help) 

In [10]:
#DSubGD算法 
def Robust_DGD_TM_robust2(a_robust2,b_robust2,agent_resource_quantity_robust2,average_resource_honest_robust2,average_cost_honest_robust2,iteration_update_robust2,dual_variable_robust2,A_robust2,iteration_robust2):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+TM+大值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust2[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust2[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust2[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust2[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust2)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust2.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust2=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust2*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust2.append(dual_variable_robust2[i][k-1]-iteration_update_robust2*((50/100)-(1/100)*agent_resource_quantity_robust2[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust2*100)),100):
            dual_variable_intermediate_robust2.append(dual_variable_robust2[i][k-1])
        #阶段二：TM鲁棒聚合 每个用户聚合自身以及邻居的对偶值    
        for i in range (0,100-int(proportion_robust2*100)):
            mix_sum_dual_robust2=0
            sort_list_robust2=[]
            sum_robust2=0
            for j in range (0,15):
                sort_list_robust2.append(dual_variable_intermediate_robust2[neighbor_robust2[i,j]])
            sort_list_robust2.sort()
            #剔除最大的q个数据和最小的q个数据
            for j in range(0,int(q_robust2[i])):
                del sort_list_robust2[14-j]
            for j in range(0,int(q_robust2[i])):
                del sort_list_robust2[0]     
            for j in range(0,len(sort_list_robust2)):
                mix_sum_dual_robust2=mix_sum_dual_robust2+sort_list_robust2[j]
            #诚实用户将剩下的数据和自身的数据做平均操作
            dual_variable_robust2[i][k]=(1/(16-2*q_robust2[i]))*(dual_variable_robust2[i][k-1]+mix_sum_dual_robust2)
        
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust2*100),100):
            dual_variable_robust2[i][k]=dual_variable_robust2[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust2*100)):
            agent_resource_quantity_robust2[i][k]=(dual_variable_robust2[i][k]-2*a_robust2[i]*b_robust2[i])/(-2*a_robust2[i])
            #原变量的投影操作
            if agent_resource_quantity_robust2[i][k] < 0:
                agent_resource_quantity_robust2[i][k]=0
            elif agent_resource_quantity_robust2[i][k] > 100: 
                agent_resource_quantity_robust2[i][k]=100   
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust2*100),100):
            agent_resource_quantity_robust2[i][k]=agent_resource_quantity_robust2[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust2=0
        for i in range (0,100-int(proportion_robust2*100)):
            sum1_robust2 = sum1_robust2 + agent_resource_quantity_robust2[i][k]
        average_resource_honest_robust2[0][k]=sum1_robust2/(100-int(proportion_robust2*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust2=0
        for i in range(0,100-int(proportion_robust2*100)):
            sum2_robust2=sum2_robust2+a_robust2[i]*(agent_resource_quantity_robust2[i][k]-b_robust2[i])*(agent_resource_quantity_robust2[i][k]-b_robust2[i])
        average_cost_honest_robust2[0][k]=sum2_robust2/(100-int(proportion_robust2*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust2[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust2[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust2[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust2[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust2)
            print("****************************************************************************")
        
        iteration_update_robust2=1/(pow(k+1,0.1)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [11]:
Robust_DGD_TM_robust2(a_robust2,b_robust2,agent_resource_quantity_robust2,average_resource_honest_robust2,average_cost_honest_robust2,iteration_update_robust2,dual_variable_robust2,A_robust2,iteration_robust2)

****************************DGD算法+TM+大值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 54.928442884415766
agent 2 在 3999 次迭代中的资源申请量为: 54.77611114579259
agent 3 在 3999 次迭代中的资源申请量为: 59.262484486973875
agent 4 在 3999 次迭代中的资源申请量为: 37.07204520871199
agent 5 在 3999 次迭代中的资源申请量为: 56.27003246352452
agent 6 在 3999 次迭代中的资源申请量为: 54.10012065726557
agent 7 在 3999 次迭代中的资源申请量为: 64.11327469963594
agent 8 在 3999 次迭代中的资源申请量为: 45.294850742233315
agent 9 在 3999 次迭代中的资源申请量为: 64.15113700929315
agent 10 在 3999 次迭代中的资源申请量为: 52.99617199660574
agent 11 在 3999 次迭代中的资源申请量为: 63.925644005127666
agent 12 在 3999 次迭代中的资源申请量为: 37.54652304579832
agent 13 在 3999 次迭代中的资源申请量为: 48.5607138986481
agent 14 在 3999 次迭代中的资源申请量为: 40.44461561761696
agent 15 在 3999 次迭代中的资源申请量为: 41.54150958088473
agent 16 在 3999 次迭代中的资源申请量为: 48.140201031902244
agent 17 在 3999 次迭代中的资源申请量为: 49.00089732839164
agent 18 在 3999 次迭代中的资源申请量为: 45.495647622179966
agent 19 在 3999 次迭代中的资源申请量为: 43.75813883789964
agent 20 在 3999 次迭代中的资源申请量为: 49.669164947772536
agent 21 在 3999 次迭代中的资源申请量为: 68.34483720495825
ag

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 55.00032811975248
agent 2 在 5999 次迭代中的资源申请量为: 54.847525941178006
agent 3 在 5999 次迭代中的资源申请量为: 59.3410001065427
agent 4 在 5999 次迭代中的资源申请量为: 37.120762070024185
agent 5 在 5999 次迭代中的资源申请量为: 56.343449658614105
agent 6 在 5999 次迭代中的资源申请量为: 54.170464131095926
agent 7 在 5999 次迭代中的资源申请量为: 64.19887060784002
agent 8 在 5999 次迭代中的资源申请量为: 45.353619147713815
agent 9 在 5999 次迭代中的资源申请量为: 64.23567617217306
agent 10 在 5999 次迭代中的资源申请量为: 53.06627087768986
agent 11 在 5999 次迭代中的资源申请量为: 64.01205020907756
agent 12 在 5999 次迭代中的资源申请量为: 37.595270606592756
agent 13 在 5999 次迭代中的资源申请量为: 48.62538394100222
agent 14 在 5999 次迭代中的资源申请量为: 40.49678816254377
agent 15 在 5999 次迭代中的资源申请量为: 41.595194363688
agent 16 在 5999 次迭代中的资源申请量为: 48.20282727098393
agent 17 在 5999 次迭代中的资源申请量为: 49.064191320652114
agent 18 在 5999 次迭代中的资源申请量为: 45.55476358436947
agent 19 在 5999 次迭代中的资源申请量为: 43.81583375859376
agent 20 在 5999 次迭代中的资源申请量为: 49.733362879436534
agent 21 在 5999 次迭代中的资源申请量为: 68.43483010956231
age

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 55.00442226445548
agent 2 在 7999 次迭代中的资源申请量为: 54.85163350303491
agent 3 在 7999 次迭代中的资源申请量为: 59.34530811420234
agent 4 在 7999 次迭代中的资源申请量为: 37.12356404749023
agent 5 在 7999 次迭代中的资源申请量为: 56.34758748232022
agent 6 在 7999 次迭代中的资源申请量为: 54.17438251009793
agent 7 在 7999 次迭代中的资源申请量为: 64.20356710341191
agent 8 在 7999 次迭代中的资源申请量为: 45.35677144268184
agent 9 在 7999 次迭代中的资源申请量为: 64.24029685263005
agent 10 在 7999 次迭代中的资源申请量为: 53.07058838687474
agent 11 在 7999 次迭代中的资源申请量为: 64.0169262759824
agent 12 在 7999 次迭代中的资源申请量为: 37.597908612822565
agent 13 在 7999 次迭代中的资源申请量为: 48.628832288526326
agent 14 在 7999 次迭代中的资源申请量为: 40.49970821825368
agent 15 在 7999 次迭代中的资源申请量为: 41.59811873181529
agent 16 在 7999 次迭代中的资源申请量为: 48.20626786264533
agent 17 在 7999 次迭代中的资源申请量为: 49.06772414208484
agent 18 在 7999 次迭代中的资源申请量为: 45.55789010999328
agent 19 在 7999 次迭代中的资源申请量为: 43.81882062717268
agent 20 在 7999 次迭代中的资源申请量为: 49.73699640872385
agent 21 在 7999 次迭代中的资源申请量为: 68.43985021032888
agent 2

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 55.004739790826626
agent 2 在 9999 次迭代中的资源申请量为: 54.85198181912552
agent 3 在 9999 次迭代中的资源申请量为: 59.3455210521573
agent 4 在 9999 次迭代中的资源申请量为: 37.123801602111925
agent 5 在 9999 次迭代中的资源申请量为: 56.34787615438801
agent 6 在 9999 次迭代中的资源申请量为: 54.17462131724452
agent 7 在 9999 次迭代中的资源申请量为: 64.20379924676494
agent 8 在 9999 次迭代中的资源申请量为: 45.356871782823966
agent 9 在 9999 次迭代中的资源申请量为: 64.24051155042528
agent 10 在 9999 次迭代中的资源申请量为: 53.07116379073131
agent 11 在 9999 次迭代中的资源申请量为: 64.01727107478807
agent 12 在 9999 次迭代中的资源申请量为: 37.59801083020241
agent 13 在 9999 次迭代中的资源申请量为: 48.62892594243436
agent 14 在 9999 次迭代中的资源申请量为: 40.49989666851355
agent 15 在 9999 次迭代中的资源申请量为: 41.59824698105755
agent 16 在 9999 次迭代中的资源申请量为: 48.20644130921519
agent 17 在 9999 次迭代中的资源申请量为: 49.06794484374164
agent 18 在 9999 次迭代中的资源申请量为: 45.55795473807037
agent 19 在 9999 次迭代中的资源申请量为: 43.818830972890524
agent 20 在 9999 次迭代中的资源申请量为: 49.73726134244498
agent 21 在 9999 次迭代中的资源申请量为: 68.44016160214473
agent

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 55.00524046714122
agent 2 在 51999 次迭代中的资源申请量为: 54.85270204126634
agent 3 在 51999 次迭代中的资源申请量为: 59.345160321276595
agent 4 在 51999 次迭代中的资源申请量为: 37.12429251047218
agent 5 在 51999 次迭代中的资源申请量为: 56.34814602997749
agent 6 在 51999 次迭代中的资源申请量为: 54.174623595150784
agent 7 在 51999 次迭代中的资源申请量为: 64.20340587952565
agent 8 在 51999 次迭代中的资源申请量为: 45.356201357304066
agent 9 在 51999 次迭代中的资源申请量为: 64.24002429423435
agent 10 在 51999 次迭代中的资源申请量为: 53.07345390827038
agent 11 在 51999 次迭代中的资源申请量为: 64.01762301274472
agent 12 在 51999 次迭代中的资源申请量为: 37.59758346399635
agent 13 在 51999 次迭代中的资源申请量为: 48.62807457252734
agent 14 在 51999 次迭代中的资源申请量为: 40.49997523200529
agent 15 在 51999 次迭代中的资源申请量为: 41.59788264010777
agent 16 在 51999 次迭代中的资源申请量为: 48.2061778616349
agent 17 在 51999 次迭代中的资源申请量为: 49.06798649640616
agent 18 在 51999 次迭代中的资源申请量为: 45.55703421022966
agent 19 在 51999 次迭代中的资源申请量为: 43.8175750750868
agent 20 在 51999 次迭代中的资源申请量为: 49.737582053486356
agent 21 在 51999 次迭代中的资源申请量为: 68

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 55.00525062577746
agent 2 在 53999 次迭代中的资源申请量为: 54.852716816534546
agent 3 在 53999 次迭代中的资源申请量为: 59.345152342278126
agent 4 在 53999 次迭代中的资源申请量为: 37.12430258052616
agent 5 在 53999 次迭代中的资源申请量为: 56.3481513306347
agent 6 在 53999 次迭代中的资源申请量为: 54.17462328323748
agent 7 在 53999 次迭代中的资源申请量为: 64.20339717585306
agent 8 在 53999 次迭代中的资源申请量为: 45.35618696611479
agent 9 在 53999 次迭代中的资源申请量为: 64.24001362495338
agent 10 在 53999 次迭代中的资源申请量为: 53.073501687812026
agent 11 在 53999 次迭代中的资源申请量为: 64.01762997330057
agent 12 在 53999 次迭代中的资源申请量为: 37.597574232990205
agent 13 在 53999 次迭代中的资源申请量为: 48.62805634865017
agent 14 在 53999 次迭代中的资源申请量为: 40.499976617608056
agent 15 在 53999 次迭代中的资源申请量为: 41.59787470882989
agent 16 在 53999 次迭代中的资源申请量为: 48.20617200468082
agent 17 在 53999 次迭代中的资源申请量为: 49.06798705033673
agent 18 在 53999 次迭代中的资源申请量为: 45.557014560623294
agent 19 在 53999 次迭代中的资源申请量为: 43.81754838365587
agent 20 在 53999 次迭代中的资源申请量为: 49.73758846706809
agent 21 在 53999 次迭代中的资源申请量为:

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 55.00526037879357
agent 2 在 55999 次迭代中的资源申请量为: 54.85273100180798
agent 3 在 55999 次迭代中的资源申请量为: 59.34514468204146
agent 4 在 55999 次迭代中的资源申请量为: 37.12431224844213
agent 5 在 55999 次迭代中的资源申请量为: 56.34815641971155
agent 6 在 55999 次迭代中的资源申请量为: 54.17462298371963
agent 7 在 55999 次迭代中的资源申请量为: 64.20338881977024
agent 8 在 55999 次迭代中的资源申请量为: 45.356173149607166
agent 9 在 55999 次迭代中的资源申请量为: 64.24000338186114
agent 10 在 55999 次迭代中的资源申请量为: 53.073547559386384
agent 11 在 55999 次迭代中的资源申请量为: 64.01763665603971
agent 12 在 55999 次迭代中的资源申请量为: 37.59756537061424
agent 13 在 55999 次迭代中的资源申请量为: 48.62803885253364
agent 14 在 55999 次迭代中的资源申请量为: 40.499977947882066
agent 15 在 55999 次迭代中的资源申请量为: 41.597867094286094
agent 16 在 55999 次迭代中的资源申请量为: 48.20616638159284
agent 17 在 55999 次迭代中的资源申请量为: 49.06798758218435
agent 18 在 55999 次迭代中的资源申请量为: 45.556995695694866
agent 19 在 55999 次迭代中的资源申请量为: 43.81752275811
agent 20 在 55999 次迭代中的资源申请量为: 49.737594624518174
agent 21 在 55999 次迭代中的资源申请量为: 6

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 55.00526975603037
agent 2 在 57999 次迭代中的资源申请量为: 54.85274464049355
agent 3 在 57999 次迭代中的资源申请量为: 59.34513731710749
agent 4 在 57999 次迭代中的资源申请量为: 37.12432154380714
agent 5 在 57999 次迭代中的资源申请量为: 56.34816131277071
agent 6 在 57999 次迭代中的资源申请量为: 54.17462269568668
agent 7 在 57999 次迭代中的资源申请量为: 64.2033807857012
agent 8 在 57999 次迭代中的资源申请量为: 45.35615986549839
agent 9 在 57999 次迭代中的资源申请量为: 64.23999353359486
agent 10 在 57999 次迭代中的资源申请量为: 53.073591663370536
agent 11 在 57999 次迭代中的资源申请量为: 64.01764308139609
agent 12 在 57999 次迭代中的资源申请量为: 37.59755684974559
agent 13 在 57999 次迭代中的资源申请量为: 48.628022030631
agent 14 在 57999 次迭代中的资源申请量为: 40.49997922689743
agent 15 在 57999 次迭代中的资源申请量为: 41.59785977317144
agent 16 在 57999 次迭代中的资源申请量为: 48.20616097516424
agent 17 在 57999 次迭代中的资源申请量为: 49.067988093571586
agent 18 在 57999 次迭代中的资源申请量为: 45.5569775577106
agent 19 在 57999 次迭代中的资源申请量为: 43.8174981200255
agent 20 在 57999 次迭代中的资源申请量为: 49.73760054468143
agent 21 在 57999 次迭代中的资源申请量为: 68.4402

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 55.00527878419711
agent 2 在 59999 次迭代中的资源申请量为: 54.852757771442754
agent 3 在 59999 次迭代中的资源申请量为: 59.34513022647929
agent 4 在 59999 次迭代中的资源申请量为: 37.124330493103095
agent 5 在 59999 次迭代中的资源申请量为: 56.34816602374171
agent 6 在 59999 次迭代中的资源申请量为: 54.174622418323345
agent 7 在 59999 次迭代中的资源申请量为: 64.20337305075395
agent 8 在 59999 次迭代中的资源申请量为: 45.356147075942886
agent 9 在 59999 次迭代中的资源申请量为: 64.23998405208313
agent 10 在 59999 次迭代中的资源申请量为: 53.07363412540962
agent 11 在 59999 次迭代中的资源申请量为: 64.01764926765934
agent 12 在 59999 次迭代中的资源申请量为: 37.59754864610771
agent 13 在 59999 次迭代中的资源申请量为: 48.62800583501489
agent 14 在 59999 次迭代中的资源申请量为: 40.49998045829696
agent 15 在 59999 次迭代中的资源申请量为: 41.59785272462672
agent 16 在 59999 次迭代中的资源申请量为: 48.206155769993956
agent 17 在 59999 次迭代中的资源申请量为: 49.06798858595063
agent 18 在 59999 次迭代中的资源申请量为: 45.55696009499556
agent 19 在 59999 次迭代中的资源申请量为: 43.8174743992087
agent 20 在 59999 次迭代中的资源申请量为: 49.73760624442476
agent 21 在 59999 次迭代中的资源申请量为: 

In [12]:
np.savetxt('./data/02_TM_oracle_agent_resource_quantity_max_attack.txt',agent_resource_quantity_robust2,fmt='%0.8f')#用户资源
np.savetxt('./data/02_TM_oracle_average_cost_honest_max_attack.txt',average_cost_honest_robust2,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/02_TM_oracle_average_resource_honest_max_attack.txt',average_resource_honest_robust2,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/02_TM_oracle_dual_variable_max_attack.txt',dual_variable_robust2,fmt='%0.8f')#用户对偶变量

# DGD+TM算法+高斯攻击1

In [13]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [14]:
#定义变量 
agent_resource_quantity_robust3 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust3=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust3=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust3 = 0.6
dual_variable_robust3 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust3= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust3=[]#绘图横坐标
A_robust3 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust3_help=[]
random_attack3_robust3= np.loadtxt("./data/random_Gaussian_attack_value_-30_5.txt")
a_robust3=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust3=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
check_robust3=np.zeros(shape=(100,60000))
q_robust3=[] #TM鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量

In [15]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust3*100)):
    dual_variable_robust3[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust3*100)),100):
    dual_variable_robust3[i][0]=random_attack3_robust3[i-94][0]  
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust3*100)):
    agent_resource_quantity_robust3[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust3=0
for i in range (0,100-int(proportion_robust3*100)):
    sum_robust3 = sum_robust3 + agent_resource_quantity_robust3[i][0]
average_resource_honest_robust3[0][0]=sum_robust3/(100-int(proportion_robust3*100))

#诚实用户平均成本初始化
sum_robust3=0
for i in range(0,100-int(proportion_robust3*100)):
    sum_robust3=sum_robust3+a_robust3[i]*(agent_resource_quantity_robust3[i][0]-b_robust3[i])*(agent_resource_quantity_robust3[i][0]-b_robust3[i])
average_cost_honest_robust3[0][0]=sum_robust3/(100-int(proportion_robust3*100))

iteration_robust3.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust3[i][j]!=0:
            neighbor_robust3_help.append(j)
neighbor_robust3=np.array(neighbor_robust3_help).reshape(100,15)

#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust3_help=0
    for j in range(0,15):
        if neighbor_robust3[i][j]==94 or neighbor_robust3[i][j]==95 or neighbor_robust3[i][j]==96 or neighbor_robust3[i][j]==97 or neighbor_robust3[i][j]==98 or neighbor_robust3[i][j]==99:
            q_robust3_help=q_robust3_help+1
    q_robust3.append(q_robust3_help) 

In [16]:
def Robust_DGD_TM_robust3(a_robust3,b_robust3,agent_resource_quantity_robust3,average_resource_honest_robust3,average_cost_honest_robust3,iteration_update_robust3,dual_variable_robust3,A_robust3,iteration_robust3):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+TM+高斯拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust3[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust3[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust3[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust3[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust3)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust3.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust3=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust3*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust3.append(dual_variable_robust3[i][k-1]-iteration_update_robust3*((50/100)-(1/100)*agent_resource_quantity_robust3[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust3*100)),100):
            dual_variable_intermediate_robust3.append(dual_variable_robust3[i][k-1])
        #阶段二：TM鲁棒聚合 每个用户聚合自身以及邻居的对偶值    
        for i in range (0,100-int(proportion_robust3*100)):
            mix_sum_dual_robust3=0
            sort_list_robust3=[]
            sum_robust3=0
            for j in range (0,15):
                sort_list_robust3.append(dual_variable_intermediate_robust3[neighbor_robust3[i,j]])
            sort_list_robust3.sort()
            #剔除最大的5个数据和最小的5个数据
            for j in range(0,int(q_robust3[i])):
                del sort_list_robust3[14-j]
            for j in range(0,int(q_robust3[i])):
                del sort_list_robust3[0]     
            for j in range(0,len(sort_list_robust3)):
                mix_sum_dual_robust3=mix_sum_dual_robust3+sort_list_robust3[j]
            #诚实用户将剩下的数据和自身的数据做平均操作
            dual_variable_robust3[i][k]=(1/(16-2*q_robust3[i]))*(dual_variable_robust3[i][k-1]+mix_sum_dual_robust3)
            
            #检测是否将拜占庭用户剔除
            for j in range(0,len(sort_list_robust3)):
                if sort_list_robust3[j]==dual_variable_robust3[94][k-1] or sort_list_robust3[j]==dual_variable_robust3[95][k-1] or sort_list_robust3[j]==dual_variable_robust3[96][k-1] or sort_list_robust3[j]==dual_variable_robust3[97][k-1] or sort_list_robust3[j]==dual_variable_robust3[98][k-1] or sort_list_robust3[j]==dual_variable_robust3[99][k-1]:
                    check_robust3[i][k]=1  
        
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust3*100),100):
            dual_variable_robust3[i][k]=random_attack3_robust3[i-94][k]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust3*100)):
            agent_resource_quantity_robust3[i][k]=(dual_variable_robust3[i][k]-2*a_robust3[i]*b_robust3[i])/(-2*a_robust3[i])
            #原变量的投影操作
            if agent_resource_quantity_robust3[i][k] < 0:
                agent_resource_quantity_robust3[i][k]=0
            elif agent_resource_quantity_robust3[i][k] > 100: 
                agent_resource_quantity_robust3[i][k]=100  
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust3*100),100):
            agent_resource_quantity_robust3[i][k]=agent_resource_quantity_robust3[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust3=0
        for i in range (0,100-int(proportion_robust3*100)):
            sum1_robust3 = sum1_robust3 + agent_resource_quantity_robust3[i][k]
        average_resource_honest_robust3[0][k]=sum1_robust3/(100-int(proportion_robust3*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust3=0
        for i in range(0,100-int(proportion_robust3*100)):
            sum2_robust3=sum2_robust3+a_robust3[i]*(agent_resource_quantity_robust3[i][k]-b_robust3[i])*(agent_resource_quantity_robust3[i][k]-b_robust3[i])
        average_cost_honest_robust3[0][k]=sum2_robust3/(100-int(proportion_robust3*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust3[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust3[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust3[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust3[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust3)
            print("****************************************************************************")
        
        iteration_update_robust3=1/(pow(k+1,0.1)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [17]:
Robust_DGD_TM_robust3(a_robust3,b_robust3,agent_resource_quantity_robust3,average_resource_honest_robust3,average_cost_honest_robust3,iteration_update_robust3,dual_variable_robust3,A_robust3,iteration_robust3)

****************************DGD算法+TM+高斯拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 54.92859191705243
agent 2 在 3999 次迭代中的资源申请量为: 54.77625907048764
agent 3 在 3999 次迭代中的资源申请量为: 59.26264780602026
agent 4 在 3999 次迭代中的资源申请量为: 37.07214611836175
agent 5 在 3999 次迭代中的资源申请量为: 56.2701848159851
agent 6 在 3999 次迭代中的资源申请量为: 54.10026678368207
agent 7 在 3999 次迭代中的资源申请量为: 64.11345274621968
agent 8 在 3999 次迭代中的资源申请量为: 45.294973223662474
agent 9 在 3999 次迭代中的资源申请量为: 64.15131291662706
agent 10 在 3999 次迭代中的资源申请量为: 52.99631625328382
agent 11 在 3999 次迭代中的资源申请量为: 63.92582329147166
agent 12 在 3999 次迭代中的资源申请量为: 37.54662456578438
agent 13 在 3999 次迭代中的资源申请量为: 48.56084874754807
agent 14 在 3999 次迭代中的资源申请量为: 40.444723951360565
agent 15 在 3999 次迭代中的资源申请量为: 41.54162131969309
agent 16 在 3999 次迭代中的资源申请量为: 48.14033128512644
agent 17 在 3999 次迭代中的资源申请量为: 49.00102878723639
agent 18 在 3999 次迭代中的资源申请量为: 45.495770974475526
agent 19 在 3999 次迭代中的资源申请量为: 43.758259437809336
agent 20 在 3999 次迭代中的资源申请量为: 49.669298118269275
agent 21 在 3999 次迭代中的资源申请量为: 68.34502412601488
agen

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 55.000336477374184
agent 2 在 5999 次迭代中的资源申请量为: 54.84753423668756
agent 3 在 5999 次迭代中的资源申请量为: 59.34100926530407
agent 4 在 5999 次迭代中的资源申请量为: 37.12076772896042
agent 5 在 5999 次迭代中的资源申请量为: 56.34345820240494
agent 6 在 5999 次迭代中的资源申请量为: 54.170472325724376
agent 7 在 5999 次迭代中的资源申请量为: 64.19888059249884
agent 8 在 5999 次迭代中的资源申请量为: 45.35362601632864
agent 9 在 5999 次迭代中的资源申请量为: 64.23568603687136
agent 10 在 5999 次迭代中的资源申请量为: 53.066278967562525
agent 11 在 5999 次迭代中的资源申请量为: 64.01206026328852
agent 12 在 5999 次迭代中的资源申请量为: 37.59527629971288
agent 13 在 5999 次迭代中的资源申请量为: 48.625391503163
agent 14 在 5999 次迭代中的资源申请量为: 40.49679423780442
agent 15 在 5999 次迭代中的资源申请量为: 41.59520062987578
agent 16 在 5999 次迭代中的资源申请量为: 48.20283457543726
agent 17 在 5999 次迭代中的资源申请量为: 49.06419869273747
agent 18 在 5999 次迭代中的资源申请量为: 45.554770501809244
agent 19 在 5999 次迭代中的资源申请量为: 43.81584052166722
agent 20 在 5999 次迭代中的资源申请量为: 49.733370347521785
agent 21 在 5999 次迭代中的资源申请量为: 68.43484059191395
agent

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 55.00442278058476
agent 2 在 7999 次迭代中的资源申请量为: 54.85163401532928
agent 3 在 7999 次迭代中的资源申请量为: 59.34530867980516
agent 4 在 7999 次迭代中的资源申请量为: 37.123564396961214
agent 5 在 7999 次迭代中的资源申请量为: 56.34758800994632
agent 6 在 7999 次迭代中的资源申请量为: 54.17438301616084
agent 7 在 7999 次迭代中的资源申请量为: 64.20356772001803
agent 8 在 7999 次迭代中的资源申请量为: 45.35677186685511
agent 9 在 7999 次迭代中的资源申请量为: 64.24029746182823
agent 10 在 7999 次迭代中的资源申请量为: 53.07058888647254
agent 11 在 7999 次迭代中的资源申请量为: 64.01692689688491
agent 12 在 7999 次迭代中的资源申请量为: 37.597908964402755
agent 13 在 7999 次迭代中的资源申请量为: 48.628832755529416
agent 14 在 7999 次迭代中的资源申请量为: 40.49970859343448
agent 15 在 7999 次迭代中的资源申请量为: 41.59811911878579
agent 16 在 7999 次迭代中的资源申请量为: 48.20626831373418
agent 17 在 7999 次迭代中的资源申请量为: 49.06772459735126
agent 18 在 7999 次迭代中的资源申请量为: 45.557890537181244
agent 19 在 7999 次迭代中的资源申请量为: 43.818821044827025
agent 20 在 7999 次迭代中的资源申请量为: 49.73699686991929
agent 21 在 7999 次迭代中的资源申请量为: 68.43985085767113
age

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 55.004739824996705
agent 2 在 9999 次迭代中的资源申请量为: 54.851981853041764
agent 3 在 9999 次迭代中的资源申请量为: 59.3455210896027
agent 4 在 9999 次迭代中的资源申请量为: 37.1238016252485
agent 5 在 9999 次迭代中的资源申请量为: 56.34787618931922
agent 6 在 9999 次迭代中的资源申请量为: 54.17462135074813
agent 7 在 9999 次迭代中的资源申请量为: 64.20379928758693
agent 8 在 9999 次迭代中的资源申请量为: 45.35687181090605
agent 9 在 9999 次迭代中的资源申请量为: 64.24051159075687
agent 10 在 9999 次迭代中的资源申请量为: 53.071163823807126
agent 11 在 9999 次迭代中的资源申请量为: 64.01727111589459
agent 12 在 9999 次迭代中的资源申请量为: 37.59801085347853
agent 13 在 9999 次迭代中的资源申请量为: 48.628925973351954
agent 14 在 9999 次迭代中的资源申请量为: 40.499896693352206
agent 15 在 9999 次迭代中的资源申请量为: 41.59824700667668
agent 16 在 9999 次迭代中的资源申请量为: 48.20644133907922
agent 17 在 9999 次迭代中的资源申请量为: 49.0679448738823
agent 18 在 9999 次迭代中的资源申请量为: 45.55795476635203
agent 19 在 9999 次迭代中的资源申请量为: 43.81883100054098
agent 20 在 9999 次迭代中的资源申请量为: 49.73726137297818
agent 21 在 9999 次迭代中的资源申请量为: 68.44016164500165
agent 

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 55.00524046714122
agent 2 在 51999 次迭代中的资源申请量为: 54.85270204126634
agent 3 在 51999 次迭代中的资源申请量为: 59.345160321276595
agent 4 在 51999 次迭代中的资源申请量为: 37.12429251047218
agent 5 在 51999 次迭代中的资源申请量为: 56.34814602997749
agent 6 在 51999 次迭代中的资源申请量为: 54.174623595150784
agent 7 在 51999 次迭代中的资源申请量为: 64.20340587952565
agent 8 在 51999 次迭代中的资源申请量为: 45.356201357304066
agent 9 在 51999 次迭代中的资源申请量为: 64.24002429423435
agent 10 在 51999 次迭代中的资源申请量为: 53.07345390827038
agent 11 在 51999 次迭代中的资源申请量为: 64.01762301274472
agent 12 在 51999 次迭代中的资源申请量为: 37.59758346399635
agent 13 在 51999 次迭代中的资源申请量为: 48.62807457252734
agent 14 在 51999 次迭代中的资源申请量为: 40.49997523200529
agent 15 在 51999 次迭代中的资源申请量为: 41.59788264010777
agent 16 在 51999 次迭代中的资源申请量为: 48.2061778616349
agent 17 在 51999 次迭代中的资源申请量为: 49.06798649640616
agent 18 在 51999 次迭代中的资源申请量为: 45.55703421022966
agent 19 在 51999 次迭代中的资源申请量为: 43.8175750750868
agent 20 在 51999 次迭代中的资源申请量为: 49.737582053486356
agent 21 在 51999 次迭代中的资源申请量为: 68

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 55.00525062577746
agent 2 在 53999 次迭代中的资源申请量为: 54.852716816534546
agent 3 在 53999 次迭代中的资源申请量为: 59.345152342278126
agent 4 在 53999 次迭代中的资源申请量为: 37.12430258052616
agent 5 在 53999 次迭代中的资源申请量为: 56.3481513306347
agent 6 在 53999 次迭代中的资源申请量为: 54.17462328323748
agent 7 在 53999 次迭代中的资源申请量为: 64.20339717585306
agent 8 在 53999 次迭代中的资源申请量为: 45.35618696611479
agent 9 在 53999 次迭代中的资源申请量为: 64.24001362495338
agent 10 在 53999 次迭代中的资源申请量为: 53.073501687812026
agent 11 在 53999 次迭代中的资源申请量为: 64.01762997330057
agent 12 在 53999 次迭代中的资源申请量为: 37.597574232990205
agent 13 在 53999 次迭代中的资源申请量为: 48.62805634865017
agent 14 在 53999 次迭代中的资源申请量为: 40.499976617608056
agent 15 在 53999 次迭代中的资源申请量为: 41.59787470882989
agent 16 在 53999 次迭代中的资源申请量为: 48.20617200468082
agent 17 在 53999 次迭代中的资源申请量为: 49.06798705033673
agent 18 在 53999 次迭代中的资源申请量为: 45.557014560623294
agent 19 在 53999 次迭代中的资源申请量为: 43.81754838365587
agent 20 在 53999 次迭代中的资源申请量为: 49.73758846706809
agent 21 在 53999 次迭代中的资源申请量为:

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 55.00526037879357
agent 2 在 55999 次迭代中的资源申请量为: 54.85273100180798
agent 3 在 55999 次迭代中的资源申请量为: 59.34514468204146
agent 4 在 55999 次迭代中的资源申请量为: 37.12431224844213
agent 5 在 55999 次迭代中的资源申请量为: 56.34815641971155
agent 6 在 55999 次迭代中的资源申请量为: 54.17462298371963
agent 7 在 55999 次迭代中的资源申请量为: 64.20338881977024
agent 8 在 55999 次迭代中的资源申请量为: 45.356173149607166
agent 9 在 55999 次迭代中的资源申请量为: 64.24000338186114
agent 10 在 55999 次迭代中的资源申请量为: 53.073547559386384
agent 11 在 55999 次迭代中的资源申请量为: 64.01763665603971
agent 12 在 55999 次迭代中的资源申请量为: 37.59756537061424
agent 13 在 55999 次迭代中的资源申请量为: 48.62803885253364
agent 14 在 55999 次迭代中的资源申请量为: 40.499977947882066
agent 15 在 55999 次迭代中的资源申请量为: 41.597867094286094
agent 16 在 55999 次迭代中的资源申请量为: 48.20616638159284
agent 17 在 55999 次迭代中的资源申请量为: 49.06798758218435
agent 18 在 55999 次迭代中的资源申请量为: 45.556995695694866
agent 19 在 55999 次迭代中的资源申请量为: 43.81752275811
agent 20 在 55999 次迭代中的资源申请量为: 49.737594624518174
agent 21 在 55999 次迭代中的资源申请量为: 6

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 55.00526975603037
agent 2 在 57999 次迭代中的资源申请量为: 54.85274464049355
agent 3 在 57999 次迭代中的资源申请量为: 59.34513731710749
agent 4 在 57999 次迭代中的资源申请量为: 37.12432154380714
agent 5 在 57999 次迭代中的资源申请量为: 56.34816131277071
agent 6 在 57999 次迭代中的资源申请量为: 54.17462269568668
agent 7 在 57999 次迭代中的资源申请量为: 64.2033807857012
agent 8 在 57999 次迭代中的资源申请量为: 45.35615986549839
agent 9 在 57999 次迭代中的资源申请量为: 64.23999353359486
agent 10 在 57999 次迭代中的资源申请量为: 53.073591663370536
agent 11 在 57999 次迭代中的资源申请量为: 64.01764308139609
agent 12 在 57999 次迭代中的资源申请量为: 37.59755684974559
agent 13 在 57999 次迭代中的资源申请量为: 48.628022030631
agent 14 在 57999 次迭代中的资源申请量为: 40.49997922689743
agent 15 在 57999 次迭代中的资源申请量为: 41.59785977317144
agent 16 在 57999 次迭代中的资源申请量为: 48.20616097516424
agent 17 在 57999 次迭代中的资源申请量为: 49.067988093571586
agent 18 在 57999 次迭代中的资源申请量为: 45.5569775577106
agent 19 在 57999 次迭代中的资源申请量为: 43.8174981200255
agent 20 在 57999 次迭代中的资源申请量为: 49.73760054468143
agent 21 在 57999 次迭代中的资源申请量为: 68.4402

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 55.00527878419711
agent 2 在 59999 次迭代中的资源申请量为: 54.852757771442754
agent 3 在 59999 次迭代中的资源申请量为: 59.34513022647929
agent 4 在 59999 次迭代中的资源申请量为: 37.124330493103095
agent 5 在 59999 次迭代中的资源申请量为: 56.34816602374171
agent 6 在 59999 次迭代中的资源申请量为: 54.174622418323345
agent 7 在 59999 次迭代中的资源申请量为: 64.20337305075395
agent 8 在 59999 次迭代中的资源申请量为: 45.356147075942886
agent 9 在 59999 次迭代中的资源申请量为: 64.23998405208313
agent 10 在 59999 次迭代中的资源申请量为: 53.07363412540962
agent 11 在 59999 次迭代中的资源申请量为: 64.01764926765934
agent 12 在 59999 次迭代中的资源申请量为: 37.59754864610771
agent 13 在 59999 次迭代中的资源申请量为: 48.62800583501489
agent 14 在 59999 次迭代中的资源申请量为: 40.49998045829696
agent 15 在 59999 次迭代中的资源申请量为: 41.59785272462672
agent 16 在 59999 次迭代中的资源申请量为: 48.206155769993956
agent 17 在 59999 次迭代中的资源申请量为: 49.06798858595063
agent 18 在 59999 次迭代中的资源申请量为: 45.55696009499556
agent 19 在 59999 次迭代中的资源申请量为: 43.8174743992087
agent 20 在 59999 次迭代中的资源申请量为: 49.73760624442476
agent 21 在 59999 次迭代中的资源申请量为: 

In [18]:
np.savetxt('./data/02_TM_oracle_agent_resource_quantity_Gaussian_attack_large.txt',agent_resource_quantity_robust3,fmt='%0.8f')#用户资源
np.savetxt('./data/02_TM_oracle_average_cost_honest_Gaussian_attack_large.txt',average_cost_honest_robust3,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/02_TM_oracle_average_resource_honest_Gaussian_attack_large.txt',average_resource_honest_robust3,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/02_TM_oracle_dual_variable_Gaussian_attack_large.txt',dual_variable_robust3,fmt='%0.8f')#用户对偶变量

# DGD+TM算法+高斯攻击2

In [19]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [20]:
#定义变量 
agent_resource_quantity_robust4 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust4=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust4=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust4 = 0.6
dual_variable_robust4 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust4= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust4=[]#绘图横坐标
A_robust4 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust4_help=[]
random_attack4_robust4= np.loadtxt("./data/random_Gaussian_attack_value_-300_40.txt")
a_robust4=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust4=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
check_robust4=np.zeros(shape=(100,60000))
q_robust4=[] #TM鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量

In [21]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust4*100)):
    dual_variable_robust4[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust4*100)),100):
    dual_variable_robust4[i][0]=random_attack4_robust4[i-94][0]
    
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust4*100)):
    agent_resource_quantity_robust4[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust4=0
for i in range (0,100-int(proportion_robust4*100)):
    sum_robust4 = sum_robust4 + agent_resource_quantity_robust4[i][0]
average_resource_honest_robust4[0][0]=sum_robust4/(100-int(proportion_robust4*100))

#诚实用户平均成本初始化
sum_robust4=0
for i in range(0,100-int(proportion_robust4*100)):
    sum_robust4=sum_robust4+a_robust4[i]*(agent_resource_quantity_robust4[i][0]-b_robust4[i])*(agent_resource_quantity_robust4[i][0]-b_robust4[i])
average_cost_honest_robust4[0][0]=sum_robust4/(100-int(proportion_robust4*100))

iteration_robust4.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust4[i][j]!=0:
            neighbor_robust4_help.append(j)
neighbor_robust4=np.array(neighbor_robust4_help).reshape(100,15)

#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust4_help=0
    for j in range(0,15):
        if neighbor_robust4[i][j]==94 or neighbor_robust4[i][j]==95 or neighbor_robust4[i][j]==96 or neighbor_robust4[i][j]==97 or neighbor_robust4[i][j]==98 or neighbor_robust4[i][j]==99:
            q_robust4_help=q_robust4_help+1
    q_robust4.append(q_robust4_help) 

In [22]:
def Robust_DGD_TM_robust4(a_robust4,b_robust4,agent_resource_quantity_robust4,average_resource_honest_robust4,average_cost_honest_robust4,iteration_update_robust4,dual_variable_robust4,A_robust4,iteration_robust4):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+TM+高斯拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust4[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust4[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust4[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust4[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust4)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust4.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust4=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust4*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust4.append(dual_variable_robust4[i][k-1]-iteration_update_robust4*((50/100)-(1/100)*agent_resource_quantity_robust4[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust4*100)),100):
            dual_variable_intermediate_robust4.append(dual_variable_robust4[i][k-1])
        #阶段二：TM鲁棒聚合 每个用户聚合自身以及邻居的对偶值    
        for i in range (0,100-int(proportion_robust4*100)):
            mix_sum_dual_robust4=0
            sort_list_robust4=[]
            sum_robust4=0
            for j in range (0,15):
                sort_list_robust4.append(dual_variable_intermediate_robust4[neighbor_robust4[i,j]])
            sort_list_robust4.sort()
            #剔除最大的4个数据和最小的4个数据
            for j in range(0,int(q_robust4[i])):
                del sort_list_robust4[14-j]
            for j in range(0,int(q_robust4[i])):
                del sort_list_robust4[0]     
            for j in range(0,len(sort_list_robust4)):
                mix_sum_dual_robust4=mix_sum_dual_robust4+sort_list_robust4[j]
            #诚实用户将剩下的数据和自身的数据做平均操作
            dual_variable_robust4[i][k]=(1/(16-2*q_robust4[i]))*(dual_variable_robust4[i][k-1]+mix_sum_dual_robust4)
            
            #检测是否将拜占庭用户剔除
            for j in range(0,len(sort_list_robust4)):
                if sort_list_robust4[j]==dual_variable_robust4[94][k-1] or sort_list_robust4[j]==dual_variable_robust4[95][k-1] or sort_list_robust4[j]==dual_variable_robust4[96][k-1] or sort_list_robust4[j]==dual_variable_robust4[97][k-1] or sort_list_robust4[j]==dual_variable_robust4[98][k-1] or sort_list_robust4[j]==dual_variable_robust4[99][k-1]:
                    check_robust4[i][k]=1
        
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust4*100),100):
            dual_variable_robust4[i][k]=random_attack4_robust4[i-94][k]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust4*100)):
            agent_resource_quantity_robust4[i][k]=(dual_variable_robust4[i][k]-2*a_robust4[i]*b_robust4[i])/(-2*a_robust4[i])
            #原变量的投影操作
            if agent_resource_quantity_robust4[i][k] < 0:
                agent_resource_quantity_robust4[i][k]=0
            elif agent_resource_quantity_robust4[i][k] > 100: 
                agent_resource_quantity_robust4[i][k]=100  
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust4*100),100):
            agent_resource_quantity_robust4[i][k]=agent_resource_quantity_robust4[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust4=0
        for i in range (0,100-int(proportion_robust4*100)):
            sum1_robust4 = sum1_robust4 + agent_resource_quantity_robust4[i][k]
        average_resource_honest_robust4[0][k]=sum1_robust4/(100-int(proportion_robust4*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust4=0
        for i in range(0,100-int(proportion_robust4*100)):
            sum2_robust4=sum2_robust4+a_robust4[i]*(agent_resource_quantity_robust4[i][k]-b_robust4[i])*(agent_resource_quantity_robust4[i][k]-b_robust4[i])
        average_cost_honest_robust4[0][k]=sum2_robust4/(100-int(proportion_robust4*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust4[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust4[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust4[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust4[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust4)
            print("****************************************************************************")
        
        iteration_update_robust4=1/(pow(k+1,0.1)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [23]:
Robust_DGD_TM_robust4(a_robust4,b_robust4,agent_resource_quantity_robust4,average_resource_honest_robust4,average_cost_honest_robust4,iteration_update_robust4,dual_variable_robust4,A_robust4,iteration_robust4)

****************************DGD算法+TM+高斯拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 57.319758690223814
agent 2 在 3999 次迭代中的资源申请量为: 57.13810584796238
agent 3 在 3999 次迭代中的资源申请量为: 61.869659901186914
agent 4 在 3999 次迭代中的资源申请量为: 38.68793832297003
agent 5 在 3999 次迭代中的资源申请量为: 58.70897374974778
agent 6 在 3999 次迭代中的资源申请量为: 56.436441465040375
agent 7 在 3999 次迭代中的资源申请量为: 66.96144369059553
agent 8 在 3999 次迭代中的资源申请量为: 47.25075427397654
agent 9 在 3999 次迭代中的资源申请量为: 66.96064081498959
agent 10 在 3999 次迭代中的资源申请量为: 55.309428426280085
agent 11 在 3999 次迭代中的资源申请量为: 66.80267697852392
agent 12 在 3999 次迭代中的资源申请量为: 39.17322805520621
agent 13 在 3999 次迭代中的资源申请量为: 50.71326772702402
agent 14 在 3999 次迭代中的资源申请量为: 42.17447222290589
agent 15 在 3999 次迭代中的资源申请量为: 43.32933506064631
agent 16 在 3999 次迭代中的资源申请量为: 50.2294145382329
agent 17 在 3999 次迭代中的资源申请量为: 51.10525046068
agent 18 在 3999 次迭代中的资源申请量为: 47.4643303264253
agent 19 在 3999 次迭代中的资源申请量为: 45.683607632827126
agent 20 在 3999 次迭代中的资源申请量为: 51.798232588813
agent 21 在 3999 次迭代中的资源申请量为: 71.33371280161958
agent 22 在

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 57.41825386646941
agent 2 在 5999 次迭代中的资源申请量为: 57.23637708533347
agent 3 在 5999 次迭代中的资源申请量为: 61.97783566223475
agent 4 在 5999 次迭代中的资源申请量为: 38.75479944878741
agent 5 在 5999 次迭代中的资源申请量为: 58.80980693000161
agent 6 在 5999 次迭代中的资源申请量为: 56.5331892937942
agent 7 在 5999 次迭代中的资源申请量为: 67.07914842432146
agent 8 在 5999 次迭代中的资源申请量为: 47.33173727266913
agent 9 在 5999 次迭代中的资源申请量为: 67.07707393604753
agent 10 在 5999 次迭代中的资源申请量为: 55.40536197516993
agent 11 在 5999 次迭代中的资源申请量为: 66.92108099972448
agent 12 在 5999 次迭代中的资源申请量为: 39.240177574939196
agent 13 在 5999 次迭代中的资源申请量为: 50.80242550639295
agent 14 在 5999 次迭代中的资源申请量为: 42.2463071900193
agent 15 在 5999 次迭代中的资源申请量为: 43.40315570567735
agent 16 在 5999 次迭代中的资源申请量为: 50.31531649068188
agent 17 在 5999 次迭代中的资源申请量为: 51.19220216199697
agent 18 在 5999 次迭代中的资源申请量为: 47.54585811470657
agent 19 在 5999 次迭代中的资源申请量为: 45.76318258268455
agent 20 在 5999 次迭代中的资源申请量为: 51.88650094874383
agent 21 在 5999 次迭代中的资源申请量为: 71.4574686082693
agent 22 在

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 57.42479971466523
agent 2 在 7999 次迭代中的资源申请量为: 57.24323560595934
agent 3 在 7999 次迭代中的资源申请量为: 61.98518766503597
agent 4 在 7999 次迭代中的资源申请量为: 38.75935206378436
agent 5 在 7999 次迭代中的资源申请量为: 58.816603621942406
agent 6 在 7999 次迭代中的资源申请量为: 56.53973565629406
agent 7 在 7999 次迭代中的资源申请量为: 67.08700202533208
agent 8 在 7999 次迭代中的资源申请量为: 47.33715174360957
agent 9 在 7999 次迭代中的资源申请量为: 67.08493577738061
agent 10 在 7999 次迭代中的资源申请量为: 55.412107694393335
agent 11 在 7999 次迭代中的资源申请量为: 66.92889641471172
agent 12 在 7999 次迭代中的资源申请量为: 39.2445399849589
agent 13 在 7999 次迭代中的资源申请量为: 50.80838596066968
agent 14 在 7999 次迭代中的资源申请量为: 42.251237706929544
agent 15 在 7999 次迭代中的资源申请量为: 43.408050853605936
agent 16 在 7999 次迭代中的资源申请量为: 50.32091390570547
agent 17 在 7999 次迭代中的资源申请量为: 51.19803024786475
agent 18 在 7999 次迭代中的资源申请量为: 47.55129107038415
agent 19 在 7999 次迭代中的资源申请量为: 45.76840188270917
agent 20 在 7999 次迭代中的资源申请量为: 51.89253482939431
agent 21 在 7999 次迭代中的资源申请量为: 71.46584164261348
agent

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 57.42529074986435
agent 2 在 9999 次迭代中的资源申请量为: 57.24399708769443
agent 3 在 9999 次迭代中的资源申请量为: 61.98586192886493
agent 4 在 9999 次迭代中的资源申请量为: 38.7597758569232
agent 5 在 9999 次迭代中的资源申请量为: 58.8171854553554
agent 6 在 9999 次迭代中的资源申请量为: 56.54031467820599
agent 7 在 9999 次迭代中的资源申请量为: 67.08761461851404
agent 8 在 9999 次迭代中的资源申请量为: 47.33758237606291
agent 9 在 9999 次迭代中的资源申请量为: 67.08561892484217
agent 10 在 9999 次迭代中的资源申请量为: 55.41289280920326
agent 11 在 9999 次迭代中的资源申请量为: 66.92944231209863
agent 12 在 9999 次迭代中的资源申请量为: 39.24480165565791
agent 13 在 9999 次迭代中的资源申请量为: 50.808859586561205
agent 14 在 9999 次迭代中的资源申请量为: 42.2517255477498
agent 15 在 9999 次迭代中的资源申请量为: 43.40840986384525
agent 16 在 9999 次迭代中的资源申请量为: 50.32124971195488
agent 17 在 9999 次迭代中的资源申请量为: 51.19850468826107
agent 18 在 9999 次迭代中的资源申请量为: 47.551709727958944
agent 19 在 9999 次迭代中的资源申请量为: 45.768741267444646
agent 20 在 9999 次迭代中的资源申请量为: 51.8931139057162
agent 21 在 9999 次迭代中的资源申请量为: 71.466581645616
agent 22 在 

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 57.42489039393356
agent 2 在 51999 次迭代中的资源申请量为: 57.245446657770486
agent 3 在 51999 次迭代中的资源申请量为: 61.9863412857717
agent 4 在 51999 次迭代中的资源申请量为: 38.76011998205433
agent 5 在 51999 次迭代中的资源申请量为: 58.81731446750085
agent 6 在 51999 次迭代中的资源申请量为: 56.54057975195992
agent 7 在 51999 次迭代中的资源申请量为: 67.08731177066737
agent 8 在 51999 次迭代中的资源申请量为: 47.33743632403383
agent 9 在 51999 次迭代中的资源申请量为: 67.08584487330761
agent 10 在 51999 次迭代中的资源申请量为: 55.41459212680618
agent 11 在 51999 次迭代中的资源申请量为: 66.92865887428125
agent 12 在 51999 次迭代中的资源申请量为: 39.24403860681701
agent 13 在 51999 次迭代中的资源申请量为: 50.8086955862205
agent 14 在 51999 次迭代中的资源申请量为: 42.25231671881697
agent 15 在 51999 次迭代中的资源申请量为: 43.408048414807524
agent 16 在 51999 次迭代中的资源申请量为: 50.32027105002229
agent 17 在 51999 次迭代中的资源申请量为: 51.198430078990555
agent 18 在 51999 次迭代中的资源申请量为: 47.5514614707927
agent 19 在 51999 次迭代中的资源申请量为: 45.7680274731715
agent 20 在 51999 次迭代中的资源申请量为: 51.89370170297432
agent 21 在 51999 次迭代中的资源申请量为: 71.46

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 57.42470685891211
agent 2 在 53999 次迭代中的资源申请量为: 57.245303331469096
agent 3 在 53999 次迭代中的资源申请量为: 61.98615949196218
agent 4 在 53999 次迭代中的资源申请量为: 38.76000864659389
agent 5 在 53999 次迭代中的资源申请量为: 58.81713817233423
agent 6 在 53999 次迭代中的资源申请量为: 56.54041363858307
agent 7 在 53999 次迭代中的资源申请量为: 67.0870962186684
agent 8 在 53999 次迭代中的资源申请量为: 47.33728935974938
agent 9 在 53999 次迭代中的资源申请量为: 67.08564296033276
agent 10 在 53999 次迭代中的资源申请量为: 55.41445832735488
agent 11 在 53999 次迭代中的资源申请量为: 66.9284317408012
agent 12 在 53999 次迭代中的资源申请量为: 39.243903288015474
agent 13 在 53999 次迭代中的资源申请量为: 50.808533718847755
agent 14 在 53999 次迭代中的资源申请量为: 42.25220186638286
agent 15 在 53999 次迭代中的资源申请量为: 43.407909535827166
agent 16 在 53999 次迭代中的资源申请量为: 50.32009743308016
agent 17 在 53999 次迭代中的资源申请量为: 51.19827405372948
agent 18 在 53999 次迭代中的资源申请量为: 47.55131134037133
agent 19 在 53999 次迭代中的资源申请量为: 45.76787079179401
agent 20 在 53999 次迭代中的资源申请量为: 51.89355759499296
agent 21 在 53999 次迭代中的资源申请量为: 71

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 57.42490479693211
agent 2 在 55999 次迭代中的资源申请量为: 57.245537035350644
agent 3 在 55999 次迭代中的资源申请量为: 61.98639488897672
agent 4 在 55999 次迭代中的资源申请量为: 38.76015507406539
agent 5 在 55999 次迭代中的资源申请量为: 58.81735136449459
agent 6 在 55999 次迭代中的资源申请量为: 56.54062095857029
agent 7 在 55999 次迭代中的资源申请量为: 67.0873362003638
agent 8 在 55999 次迭代中的资源申请量为: 47.337455692201424
agent 9 在 55999 次迭代中的资源申请量为: 67.08589064543908
agent 10 在 55999 次迭代中的资源申请量为: 55.41469203709096
agent 11 在 55999 次迭代中的资源申请量为: 66.92866377191136
agent 12 在 55999 次迭代中的资源申请量为: 39.24402821621155
agent 13 在 55999 次迭代中的资源申请量为: 50.80871677945921
agent 14 在 55999 次迭代中的资源申请量为: 42.252363525316596
agent 15 在 55999 次迭代中的资源申请量为: 43.40805668573291
agent 16 在 55999 次迭代中的资源申请量为: 50.320257733213516
agent 17 在 55999 次迭代中的资源申请量为: 51.198454255032914
agent 18 在 55999 次迭代中的资源申请量为: 47.55147680997856
agent 19 在 55999 次迭代中的资源申请量为: 45.76802306333965
agent 20 在 55999 次迭代中的资源申请量为: 51.89375352128787
agent 21 在 55999 次迭代中的资源申请量为: 

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 57.42492086297845
agent 2 在 57999 次迭代中的资源申请量为: 57.24558881041884
agent 3 在 57999 次迭代中的资源申请量为: 61.98643030402336
agent 4 在 57999 次迭代中的资源申请量为: 38.76017788539109
agent 5 在 57999 次迭代中的资源申请量为: 58.817378229892576
agent 6 在 57999 次迭代中的资源申请量为: 56.54064946519913
agent 7 在 57999 次迭代中的资源申请量为: 67.08735879421086
agent 8 在 57999 次迭代中的资源申请量为: 47.33747244116454
agent 9 在 57999 次迭代中的资源申请量为: 67.08592315662602
agent 10 在 57999 次迭代中的资源申请量为: 55.414748074660714
agent 11 在 57999 次迭代中的资源申请量为: 66.9286772460828
agent 12 在 57999 次迭代中的资源申请量为: 39.24402964591886
agent 13 在 57999 次迭代中的资源申请量为: 50.80873515802286
agent 14 在 57999 次迭代中的资源申请量为: 42.25239231517265
agent 15 在 57999 次迭代中的资源申请量为: 43.40806754005555
agent 16 在 57999 次迭代中的资源申请量为: 50.32025957474891
agent 17 在 57999 次迭代中的资源申请量为: 51.19847382819267
agent 18 在 57999 次迭代中的资源申请量为: 47.55149171559686
agent 19 在 57999 次迭代中的资源申请量为: 45.768028496941476
agent 20 在 57999 次迭代中的资源申请量为: 51.8937862185708
agent 21 在 57999 次迭代中的资源申请量为: 71.

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 57.42491635879773
agent 2 在 59999 次迭代中的资源申请量为: 57.24561883726861
agent 3 在 59999 次迭代中的资源申请量为: 61.9864425182172
agent 4 在 59999 次迭代中的资源申请量为: 38.76018632514667
agent 5 在 59999 次迭代中的资源申请量为: 58.81738367977727
agent 6 在 59999 次迭代中的资源申请量为: 56.5406573304148
agent 7 在 59999 次迭代中的资源申请量为: 67.08735668963004
agent 8 在 59999 次迭代中的资源申请量为: 47.337472155786756
agent 9 在 59999 次迭代中的资源申请量为: 67.08593088800438
agent 10 在 59999 次迭代中的资源申请量为: 55.41478269323279
agent 11 在 59999 次迭代中的资源申请量为: 66.92866619230585
agent 12 在 59999 次迭代中的资源申请量为: 39.2440174186536
agent 13 在 59999 次迭代中的资源申请量为: 50.80873478487513
agent 14 在 59999 次迭代中的资源申请量为: 42.25240551758233
agent 15 在 59999 次迭代中的资源申请量为: 43.408063017862254
agent 16 在 59999 次迭代中的资源申请量为: 50.32024389303178
agent 17 在 59999 次迭代中的资源申请量为: 51.19847505719402
agent 18 在 59999 次迭代中的资源申请量为: 47.55148953933224
agent 19 在 59999 次迭代中的资源申请量为: 45.76801757008577
agent 20 在 59999 次迭代中的资源申请量为: 51.89379985442934
agent 21 在 59999 次迭代中的资源申请量为: 71.46

In [24]:
np.savetxt('./data/02_TM_oracle_agent_resource_quantity_Gaussian_attack_small.txt',agent_resource_quantity_robust4,fmt='%0.8f')#用户资源
np.savetxt('./data/02_TM_oracle_average_cost_honest_Gaussian_attack_small.txt',average_cost_honest_robust4,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/02_TM_oracle_average_resource_honest_Gaussian_attack_small.txt',average_resource_honest_robust4,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/02_TM_oracle_dual_variable_Gaussian_attack_small.txt',dual_variable_robust4,fmt='%0.8f')#用户对偶变量

In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
W=np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt") #双随机的网络拓扑矩阵W（对角线处非零）
#将矩阵W中所有非零的元素赋值为1 方便后续的计算
for i in range(0,100):
    for j in range(0,100):
        if W[i][j]!=0:
            W[i][j]=1
W_honest=np.empty([94,94],dtype=float)
#在通信网络中剔除6个拜占庭用户  6个拜占庭用户：编号94-99
for i in range(0,94):
    for j in range(0,94):
        if W[i][j]!=0:
            W_honest[i][j]=1
        else:
            W_honest[i][j]=0
#将矩阵W_honest设置成行随机矩阵
degree=[]
for i in range(0,94):
    d=0
    for j in range(0,94):
        if W_honest[i][j]!=0:
            d=d+1
    degree.append(d)

for i in range(0,94):
    for j in range(0,94):
        if W_honest[i][j]!=0:
            W_honest[i][j]=(1/degree[i])

In [4]:
W_honest_mat=np.matrix(W_honest)

In [5]:
W_honest_mat

matrix([[0.07142857, 0.07142857, 0.        , ..., 0.07142857, 0.07142857,
         0.        ],
        [0.0625    , 0.0625    , 0.0625    , ..., 0.        , 0.        ,
         0.0625    ],
        [0.        , 0.0625    , 0.0625    , ..., 0.0625    , 0.        ,
         0.        ],
        ...,
        [0.0625    , 0.        , 0.0625    , ..., 0.0625    , 0.0625    ,
         0.        ],
        [0.0625    , 0.        , 0.        , ..., 0.0625    , 0.0625    ,
         0.0625    ],
        [0.        , 0.06666667, 0.        , ..., 0.        , 0.06666667,
         0.06666667]])

In [6]:
W_honest_mat_T=(W_honest_mat).T

In [7]:
#计算W_honest^{T}*1
a=[]
for i in range(0,94):
    k=0
    for j in range(0,94):
        k=k+W_honest_mat_T[i,j]
    a.append(k)

In [8]:
b=[]
for i in range(0,94):
    b.append(a[i]-1)

In [9]:
sum=0
for i in range(0,94):
    sum=sum+(abs(b[i])*abs(b[i]))

In [10]:
sum

0.2947443067463671

In [11]:
sum/94

0.0031355777313443307